# Self-adaptive oversampling method 

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_recall_fscore_support,roc_curve,auc,accuracy_score,classification_report, confusion_matrix
from scipy.stats.mstats import gmean
from sklearn.datasets import make_circles
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
from sklearn.preprocessing import Normalizer,MinMaxScaler,StandardScaler
import numpy as np
import os
import pandas as pd
import numpy.matlib
from numpy import random
import os
import math

from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings("ignore")



In [2]:
def fit_boosting(X, y, n_estimators=20):
    n_samples, n_features = X.shape
    D = np.ones((n_samples, ))
    estimators = []

    for t in range(n_estimators):
        D = D / np.sum(D)                   

        h = DecisionTreeClassifier(max_depth=5)  
        h.fit(X, y, sample_weight=D)        

        ypred = h.predict(X)   
        e = 1 - accuracy_score(y, ypred, sample_weight=D)  
        if e==0.0:
            e=0.000000000001
        if e==1.0:
            e=0.9999
        a = 0.5 * np.log((1 - e) / e)               

        m = (y == ypred) * 1 + (y != ypred) * -1    
        D *= np.exp(-a * m)
                 
        estimators.append((a, h))  

    return estimators
def predict_boosting(X, estimators):
    pred = np.zeros((X.shape[0], ))
    y_proba = np.zeros((X.shape[0], ))

    for a, h in estimators:
        pred += a * h.predict(X)
        y_proba+= h.predict_proba(X)[:, 1] 

    y = np.sign(pred)

    return y , (y_proba/len(estimators))

In [3]:
def gnerandsphere(center,d,N,r):
    random_directions = random.randn(N,d);
    s=np.sqrt(np.array(np.sum(random_directions**2,axis=1)))
    random_directions = random_directions/np.matlib.repmat(s,d,1).T; 
    random_radii =r*random.rand(N,1);
    Y = random_directions* np.matlib.repmat(random_radii,1,d);
    Y = np.matlib.repmat(center,N,1)+Y;
    return Y

In [4]:
def unionSphere(data,dataradious):
    pd = L2_dist(data,data);
    pddist =np.copy(pd);
    temp=[(v,i) for i,v in enumerate(dataradious[:,-1])]
    temp.sort(reverse=True)
    [dummy,sortid]=zip(*temp)
    sortdataradious = dataradious[sortid,:]
    #print("sortdataradious",sortdataradious )
    flag =np.zeros((len(data),1))-1
    for i in range(len(sortdataradious)):
        if flag[i] == -1:
            flag[i] = sortdataradious[i,0];
            leftindex =[]
            for j in range(i+1,len(sortdataradious)):
                if flag[j]==-1:
                    leftindex.append(j)
            
            leftunhandledid = sortdataradious[leftindex,0];
            iindex=sortdataradious[i,0]
            leftdist=[]
            for k in range(len(leftunhandledid)):
                jindex=leftunhandledid[k]
                leftdist.append(pd[iindex,jindex])
            leftdist=np.array(leftdist)
            includedset =np.argwhere(leftdist <= sortdataradious[i,2]).tolist();
            if len(includedset)>0:
                for k in range(len(includedset)):
                    in_id =includedset[k][0]
                    flag[leftindex[in_id]] = flag[i];

    belongto =np.zeros((len(dataradious),1));
    belongto[sortdataradious[:,0].tolist()] = flag;
    dataradious=dataradious.tolist()
    for k in range(len(dataradious)):
        dataradious[k].append(int(belongto[k,0]))
    dataradious=np.array(dataradious)    
    return [dataradious,pddist]

In [5]:
def calradious(i,ilabel,majlabel,minlabel,minENNdist,minenemykneigor,majENNdist,majenemykneigor):
    IR = len(majENNdist)/len(minENNdist);
    tha = 1.0/(1+np.exp(-(IR-1)));
    if ilabel == minlabel:
        nej = int(minenemykneigor[i][0]);
        dij = minENNdist[i][0];
        jlabel =  majlabel;
        ENN=int(majenemykneigor[nej][0])
        if ( i == ENN):
            dradius = dij*tha;
            return dradius ;
        else:
            dt = calradious(nej,jlabel,majlabel,minlabel,minENNdist,minenemykneigor,majENNdist,majenemykneigor);
            dradius = dij-dt;
            return dradius ;
    else:
        nej = int(majenemykneigor[i][0]);
        dij = majENNdist[i][0];
        jlabel =  minlabel;
        ENN=int(minenemykneigor[nej][0])
        if ( i == ENN):
            dradius = dij*(1-tha);
            return dradius ;
        else:
            dt = calradious(nej,jlabel,majlabel,minlabel,minENNdist,minenemykneigor,majENNdist,majenemykneigor);
            dradius = dij-dt;
            return dradius ;


In [6]:
def L2_dist(Min_data,Maj_data):
    x = Min_data;
    xsup = Maj_data;
    ps = np.dot(x,xsup.T); 
    [nps,pps]=ps.shape;
    normx =np.array([np.sum(x**2, axis=1)]).T;
    normxsup = np.sum(xsup**2, axis=1);
    A=np.matlib.repmat(normx,1,pps)
    B=np.matlib.repmat(normxsup.T,nps,1)
    ps = -2*ps + A + B; 
    E=np.sqrt(ps);
    return E

In [7]:
def kNNenemyPoint(Min_data,Maj_data):
    k_nNum = 1;
    NT = len(Min_data)
    NTNT = len(Maj_data)
    minENNdist=np.ones((NT,k_nNum))*math.inf;
    minenemykneigor=np.ones((NT,k_nNum))*math.inf;
    majENNdist= np.ones((NTNT,k_nNum))*math.inf;
    majenemykneigor =np.ones((NTNT,k_nNum))*math.inf;
    pd = L2_dist(Min_data,Maj_data);
    for i in range(NT):
        d=np.copy(pd[i,:])
        min_id=[]
        for k in range(k_nNum):
            tmp=np.min(d);
            index=np.argmin(d)
            d[index]=math.inf;
            min_id.append(index)
        minENNdist[i,:] = pd[i, min_id];
        minenemykneigor[i,:] =  min_id
    pd = pd.T
    majmindistmax = pd;
    for i in range(NTNT):   
        d=np.copy(pd[i,:])
        min_id=[];
        for k in range(k_nNum):
            tmp=np.min(d);
            index=np.argmin(d)
            d[index]=math.inf;
            min_id.append(index)
        majENNdist[i,:] = pd[i,min_id];
        majenemykneigor[i,:] = min_id
    return [minENNdist,minenemykneigor,majENNdist,majenemykneigor,majmindistmax]

In [8]:
#data_name='vehicle0.dat'
data_name='wisconsin.dat'
Size_data=846
C_maj=647
F_maj=2
C_min=199
F_min=2
head=0 # number of line without sample (header)

if data_name=='glass1.dat':
    Size_data=214
    C_maj=138
    F_maj=9
    C_min=76
    F_min=9
    head=14 # index of first data sample

if data_name=='ecoli1.dat':
    Size_data=336
    C_maj=259
    F_maj=7
    C_min=77
    F_min=7
    head=12 # index of first data sample
if data_name=='ecoli4.dat':
    Size_data=336
    C_maj=316
    F_maj=7
    C_min=20
    F_min=7
    head=12 # index of first data sample

if data_name=='glass4.dat':
    Size_data=214
    C_maj=201
    F_maj=9
    C_min=13
    F_min=9
    head=14 # index of first data sample
if data_name=='yeast-2_vs_8.dat':
    Size_data=482
    C_maj=462
    F_maj=8
    C_min=20
    F_min=8
    head=13 # index of first data sample

if data_name=='yeast4.dat':
    Size_data=1484
    C_maj=1433
    F_maj=8
    C_min=51
    F_min=8
    head=13 # index of first data sample
if data_name=='yeast-1-2-8-9_vs_7.dat':
    Size_data=947
    C_maj=917
    F_maj=8
    C_min=30
    F_min=8
    head=13 # index of first data sample

if data_name=='yeast5.dat':
    Size_data=1484
    C_maj=1440
    F_maj=8
    C_min=44
    F_min=8
    head=13 # index of first data sample 

if data_name=='abalone9-18.dat':
    Size_data=731
    C_maj=689
    F_maj=8
    C_min=42
    F_min=8
    head=13 # index of first data sample 

   
    
if data_name=='page-blocks-1-3_vs_4.dat':
    Size_data=472
    C_maj=444
    F_maj=10
    C_min=28
    F_min=10
    head=15 # index of first data sample 


if data_name=='glass-0-1-2-3_vs_4-5-6.dat':
    Size_data=214
    C_maj=163
    F_maj=9
    C_min=51
    F_min=9
    head=14 # index of first data sample  
    
if data_name=='glass0.dat':
    Size_data=214
    C_maj=144
    F_maj=9
    C_min=70
    F_min=9
    head=14 # index of first data sample     
  
if data_name=='vehicle3.dat':
    Size_data=846
    C_maj=634
    F_maj=18
    C_min=212
    F_min=18
    head=23 # index of first data sample     
if data_name=='ecoli2.dat':
    Size_data=336
    C_maj=284
    F_maj=7
    C_min=52
    F_min=7
    head=12 # index of first data sample 
if data_name=='yeast6.dat':
    Size_data=1484
    C_maj=1449
    F_maj=8
    C_min=35
    F_min=8
    head=13 # index of first data sample   
  
if data_name=='yeast-2_vs_4.dat':
    Size_data=514
    C_maj=463
    F_maj=8
    C_min=51
    F_min=8
    head=13 # index of first data sample    

if data_name=='yeast3.dat':
    Size_data=1484
    C_maj=1321
    F_maj=8
    C_min=163
    F_min=8
    head=13 # index of first data sample     

    
if data_name=='yeast-1-4-5-8_vs_7.dat':
    Size_data=693
    C_maj=663
    F_maj=8
    C_min=30
    F_min=8
    head=13 # index of first data sample     
if data_name=='segment0.dat':
    Size_data=2308
    C_maj=1979
    F_maj=19
    C_min=329
    F_min=19
    head=24 # index of first data sample 

if data_name=='yeast-0-5-6-7-9_vs_4.dat':
    Size_data=528
    C_maj=477
    F_maj=8
    C_min=51
    F_min=8
    head=13 # index of first data sample 

if data_name=='ecoli3.dat':
    Size_data=336
    C_maj=301
    F_maj=7
    C_min=35
    F_min=7
    head=12 # index of first data sample 
if data_name=='vowel0.dat':
    Size_data=988
    C_maj=898
    F_maj=13
    C_min=90
    F_min=13
    head=18 # index of first data sample 

if data_name=='glass2.dat':
    Size_data=214
    C_maj=197
    F_maj=9
    C_min=17
    F_min=9
    head=14 # index of first data sampl
if data_name=='glass5.dat':
    Size_data=214
    C_maj=205
    F_maj=9
    C_min=9
    F_min=9
    head=14 # index of first data sample 
    
if data_name=='glass6.dat':
    Size_data=214
    C_maj=185
    F_maj=9
    C_min=29
    F_min=9
    head=14 # index of first data sample 

if data_name=='new-thyroid1.dat':
    Size_data=215
    C_maj=180
    F_maj=5
    C_min=35
    F_min=5
    head=10 # index of first data sample
    
if data_name=='ecoli-0_vs_1.dat':
    Size_data=220
    C_maj=77
    F_maj=7
    C_min=143
    F_min=7
    head=12 # index of first data sample 
if data_name=='wisconsin.dat':
    Size_data=683
    C_maj=444
    F_maj=9
    C_min=239
    F_min=9
    head=14 # index of first data sample 
if data_name=='iris0.dat':
    Size_data=150
    C_maj=100
    F_maj=4
    C_min=50
    F_min=4
    head=9 # index of first data sample 
    
if data_name=='haberman.dat':
    Size_data=306
    C_maj=225
    F_maj=3
    C_min=81
    F_min=3
    head=8 # index of first data sample   
if data_name=='abalone19.dat':
    Size_data=4174
    C_maj=4142
    F_maj=8
    C_min=32
    F_min=8
    head=13 # index of first data sample  
    
if data_name=='kddcup-rootkit-imap_vs_back.dat':
    Size_data=2225
    C_maj=2203
    F_maj=41
    C_min=22
    F_min=41
    head=46 # index of first data sample 
if data_name=='penbased.dat':
    Size_data=306
    C_maj=286
    F_maj=16
    C_min=20
    F_min=16
    head=21 # index of first data sample 
    
if data_name=='autos.dat':
    Size_data=159
    C_maj=156
    F_maj=25
    C_min=3
    F_min=25
    head=30 # index of first data sample 
 
if data_name=='ecoli.dat':
    Size_data=336
    C_maj=334
    F_maj=7
    C_min=2
    F_min=7
    head=12 # index of first data sample
if data_name=='pageblocks.dat':
    Size_data=548
    C_maj=334
    F_maj=10
    C_min=3
    F_min=10
    head=15 # index of first data sample
    
data_maj = np.zeros((C_maj,F_maj))
labels_maj = np.zeros((C_maj,))
data_min = np.zeros((C_min,F_min))
labels_min = np.zeros((C_min,))

f1=0
f2=0
'''v1=['icmp', 'tcp', 'udp']
v3=['OTH', 'REJ', 'RSTO', 'RSTOS0', 'RSTR', 'S0', 'S1', 'S2', 'S3', 'SF', 'SH']
v2=['auth', 'bgp', 'courier', 'csnet_ns', 'ctf', 'daytime', 'discard', 'domain', 'domain_u', 'echo', 
    'eco_i', 'ecr_i', 'efs', 'exec', 'finger', 'ftp', 'ftp_data', 'gopher', 'hostnames', 'http', 
    'http_443', 'imap4', 'IRC', 'iso_tsap', 'klogin', 'kshell', 'ldap', 'link', 'login', 'mtp', 'name', 
    'netbios_dgm', 'netbios_ns', 'netbios_ssn', 'netstat', 'nnsp', 'nntp', 'ntp_u', 'other', 'pm_dump', 'pop_2', 'pop_3',
    'printer', 'private', 
    'red_i', 'remote_job', 'rje', 'shell', 'smtp', 'sql_net', 'ssh', 'sunrpc', 'supdup', 'systat', 'telnet', 'tftp_u', 'time', 'tim_i', 'urh_i', 'urp_i', 'uucp', 'uucp_path', 'vmnet', 'whois', 'X11', 'Z39_50']
with open( data_name,'r') as file:
    a = file.readlines()
    print(a[0:head])
    for i in range(Size_data):
        a[i+head] = a[i+head].replace(" ","")
        b = a[i+head].split(',')
        print(i,b)
        #b[1]=v1.index(b[1]) 
        #b[3]=v3.index(b[3]) 
        #b[2]=v2.index(b[2]) 
        if b[0]=='M':
            b[0]=0
        if b[0]=='F':
            b[0]=1
        if b[0]=='I':
            b[0]=2
        if b[F_maj].rfind("positive")!=(-1):
            b[F_maj] = -1
            data_min[f2,:] = np.array(b[:F_maj])
            labels_min[f2] = b[F_maj] 
            f2+=1
        elif b[F_maj].rfind("negative")!=(-1):
            b[F_maj] = 1
            data_maj[f1,:] = np.array(b[:F_maj])
            labels_maj[f1] = b[F_maj] 
            f1+=1
        if int(b[F_maj])==3:
            b[F_maj] = -1
            data_min[f2,:] = np.array(b[:F_maj])
            labels_min[f2] = b[F_maj] 
            f2+=1
        else:
            b[F_maj] = 1
            data_maj[f1,:] = np.array(b[:F_maj])
            labels_maj[f1] = b[F_maj] 
            f1+=1
            

print(f1,f2)'''

with open( data_name,'r') as file:
    a = file.readlines()
    print(a[0:head])
    for i in range(Size_data):
        a[i+head] = a[i+head].replace(" ","")
        b = a[i+head].split(',')
        print(i,b)
        if b[0]=='M':
            b[0]=0
        if b[0]=='F':
            b[0]=1
        if b[0]=='I':
            b[0]=2
        if b[F_maj].rfind("positive")!=(-1):
            b[F_maj] = -1
            data_min[f2,:] = np.array(b[:F_maj])
            labels_min[f2] = b[F_maj] 
            f2+=1
        elif b[F_maj].rfind("negative")!=(-1):
            b[F_maj] = 1
            data_maj[f1,:] = np.array(b[:F_maj])
            labels_maj[f1] = b[F_maj] 
            f1+=1
print(f1,f2)



Xtrn_maj, Xtst_maj, ytrn_maj, ytst_maj = train_test_split(data_maj, labels_maj, test_size=0.25, random_state=13)
Xtrn_min, Xtst_min, ytrn_min, ytst_min = train_test_split(data_min, labels_min, test_size=0.25, random_state=13)
scl = StandardScaler()
data_train=scl.fit_transform(np.vstack((Xtrn_min,Xtrn_maj)))
data_test = scl.transform(np.vstack((Xtst_min,Xtst_maj)))
features_min=data_train[0:len(Xtrn_min),:]
features_maj=data_train[len(Xtrn_min):,:] 

['@relation wisconsin\n', '@attribute ClumpThickness integer [1, 10]\n', '@attribute CellSize integer [1, 10]\n', '@attribute CellShape integer [1, 10]\n', '@attribute MarginalAdhesion integer [1, 10]\n', '@attribute EpithelialSize integer [1, 10]\n', '@attribute BareNuclei integer [1, 10]\n', '@attribute BlandChromatin integer [1, 10]\n', '@attribute NormalNucleoli integer [1, 10]\n', '@attribute Mitoses integer [1, 10]\n', '@attribute Class {positive,negative}\n', '@inputs ClumpThickness, CellSize, CellShape, MarginalAdhesion, EpithelialSize, BareNuclei, BlandChromatin, NormalNucleoli, Mitoses\n', '@outputs Class\n', '@data\n']
0 ['5', '1', '1', '1', '2', '1', '3', '1', '1', 'negative\n']
1 ['5', '4', '4', '5', '7', '10', '3', '2', '1', 'negative\n']
2 ['3', '1', '1', '1', '2', '2', '3', '1', '1', 'negative\n']
3 ['6', '8', '8', '1', '3', '4', '3', '7', '1', 'negative\n']
4 ['4', '1', '1', '3', '2', '1', '3', '1', '1', 'negative\n']
5 ['8', '10', '10', '8', '7', '10', '9', '7', '1', 

In [9]:
def Oversampling(oversamplesize,numofsphere_min,uncoveredpoint,finalminradius,finalmincenters ):
    dim=len(finalmincenters[0])
    alpha = 1;
    beta = 1;
    pi = np.array(np.exp(-alpha*finalminradius));
    pj = np.array(np.exp(-beta*numofsphere_min));
    pi =np.array((pi+pj)/np.sum(pi+pj)) 
    suboversampesizes = np.ceil(pi*oversamplesize)
    generatedX = [];
    for i in range(len(uncoveredpoint)):
        X = gnerandsphere(finalmincenters[i,:],dim,int(suboversampesizes[i]),finalminradius[i]); 
        generatedX.append(X);
    return generatedX,suboversampesizes

In [10]:
features_min=data_min
features_maj=data_maj
dim=F_maj
IMratio = len(features_maj)/len(features_min)
[minENNdist,minenemykneigor,majENNdist,majenemykneigor,majmindistmax]=kNNenemyPoint(features_min,features_maj);
majlabel = 1;
minlabel = -1;
mindradious = [];
for i in range(len(features_min)):
    ilabel = minlabel;
    dradius = calradious(i,ilabel,majlabel,minlabel,minENNdist,minenemykneigor,majENNdist,majenemykneigor);
    mindradious.append([i,features_min[i,:],dradius]);
    #mindradious.append([i,dradius]);
mindradious=np.array(mindradious)
data = np.copy(features_min)
[mindradious,mindist] = unionSphere(data,mindradious);
uncoveredpoint_min = list(set(mindradious[:,-1]));
numofsphere_min = [];
for i in range(len(uncoveredpoint_min)):
        j = uncoveredpoint_min[i];
        numofsphere_min.append(np.sum(mindradious[:,-1]==j))
numofsphere_min=np.array(numofsphere_min)
finalmincenters = np.array(mindradious[uncoveredpoint_min,1].tolist());
finalminradius  = np.array(mindradious[uncoveredpoint_min,2].tolist()); 


In [11]:
complexitymeasure = 1;
oversamplesize = math.ceil(complexitymeasure*(len(features_maj)-len(features_min)));
print(oversamplesize)
X_syn,_=Oversampling(oversamplesize,numofsphere_min,uncoveredpoint_min,finalminradius,finalmincenters )

205


In [12]:
X_syns=np.vstack(X_syn)
print(len(X_syns))
data_min=np.vstack((X_syns,data_min))
data=np.vstack((data_min,data_maj))

y_syns = np.full(X_syns.shape[0], fill_value=labels_min[0], dtype=np.int64)
y_min=np.hstack((y_syns,labels_min))
labels=np.hstack((y_min,labels_maj))

Xtrn, Xtst, ytrn, ytst = train_test_split(data, labels, test_size=0.25, random_state=13)
scl = StandardScaler()
data_train=scl.fit_transform(Xtrn)
data_test = scl.transform(Xtst)


233


In [13]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5,random_state=76,shuffle=True) # 5fold cross validation
k_g_mean=[]
k_roc=[]
k_PR=[]
scl = StandardScaler()
for train_index, test_index in kf.split(data):

        data_train=scl.fit_transform(data[train_index])
        data_test=scl.transform(data[test_index])
        ytrn=labels[train_index]
        ytst=labels[test_index]
        estimators = fit_boosting(data_train, ytrn, 150)
        predictions, y_proba=predict_boosting(data_test, estimators)
        #print("Confusion Matrix:")
        #print(confusion_matrix(ytst, predictions))
        # print()
        #print("Classification Report")
        #print(classification_report(ytst, predictions))
        roc_auc= roc_auc_score(ytst, y_proba)
        precision, recall, thresholds = precision_recall_curve(ytst, y_proba)
        PR_auc_score = auc(recall, precision)
        g_mean = 1.0
        for label in np.unique(ytst):
            idx = (ytst== label)
            ytest=np.array(ytst)
            preds=np.array(predictions)
            g_mean *= accuracy_score(ytest[idx], predictions[idx])
        g_mean = np.sqrt(g_mean)
        k_g_mean.append(g_mean)
        k_roc.append(roc_auc)
        k_PR.append(PR_auc_score)

    

print("G_means: ", k_g_mean, "Mean:",np.array(k_g_mean).mean())
print("roc_auc: ", k_roc, "Mean:",np.array(k_roc).mean())
print("PR_auc_score: ", k_PR, "Mean:",np.array(k_PR).mean())

G_means:  [0.9848857801796105, 0.9789450103725609, 0.9854006762498304, 0.9777588819690793, 0.9775252199076787] Mean: 0.9809031137357518
roc_auc:  [0.9979761904761905, 0.9925766283524904, 0.9938977879481312, 0.9983265598852499, 0.9989247311827957] Mean: 0.9963403795689715
PR_auc_score:  [0.9984303286803288, 0.9952200153813919, 0.992886205047034, 0.998323364899985, 0.998964144744263] Mean: 0.9967648117506005


In [ ]:
from sklearn.metrics import accuracy_score
estimators = fit_boosting(data_train, ytrn, 10)
predictions, y_proba=predict_boosting(data_test, estimators)
print("Confusion Matrix:")
print(confusion_matrix(ytst, predictions))
print()
print("Classification Report")
print(classification_report(ytst, predictions))
roc_auc= roc_auc_score(ytst, y_proba)
precision, recall, thresholds = precision_recall_curve(ytst, y_proba)
PR_auc_score = auc(recall, precision)
g_mean = 1.0
for label in np.unique(ytst):
    idx = (ytst== label)
    ytest=np.array(ytst)
    preds=np.array(predictions)
    g_mean *= accuracy_score(ytest[idx], predictions[idx])
g_mean = np.sqrt(g_mean)
print("G_means: ", g_mean)
print("ROC AUC: ", roc_auc)
print("PR AUC: ", PR_auc_score)
predictions
